In [4]:
pip install konlpy


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install kiwipiepy

Note: you may need to restart the kernel to use updated packages.


In [2]:
from tqdm import tqdm

In [3]:
import sys
import os
import pandas as pd

sys.path.append('/app/EZPZ_Project/modules/torchmodules') # 토치 모델 뉴스 요약 및 감정평가 가져오기

In [4]:
import pymysql
import cryptography
import sql_connection as sc #mysql connection

In [5]:
from privates.ezpz_db import * #db연동 정보

In [6]:
#토치 모듈 키워드추출 토크나이저와 t5 base sum 리뷰 요약
#import tokenizer 토크나이저는 일단 나중에
from service_models import ServiceModels #t5model

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
data_check= ServiceModels() #모델 서빙 모듈 객체

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [107]:
"""def get_all_comp_uid():
    comp_list = []
    sql = ' select comp_uid from comp_info where is_reged = "Y" ' #기업정보 크롤링 처리된 회사들 이름 전부 가져옴
    comp_temp_list = sc.conn_and_exec(sql)
    comp_uid_list = list(comp_temp_list)
    return comp_uid_list"""

'def get_all_comp_uid():\n    comp_list = []\n    sql = \' select comp_uid from comp_info where is_reged = "Y" \' #기업정보 크롤링 처리된 회사들 이름 전부 가져옴\n    comp_temp_list = sc.conn_and_exec(sql)\n    comp_uid_list = list(comp_temp_list)\n    return comp_uid_list'

1. 분기별로 데이터를 나눠서 정보 가져오는 함수: data_split_by_quater() return 분기별 리뷰 리스트/ 03(1분기) 04(2분기) 05(3분기) 06(4분기)
2. 반기별로 데이터를 나눠서 정보 가져오는 함수: data_split_by_half() return 반기별 리뷰 리스트/ 01(전반기) 02(하반기)
3. 모든 기간 데이터로 정보 가져오는 함수 data_not_split 00: 모든 기간 return 모든기간 리뷰 리스트(해당 회사 전부 리뷰 요약)
4. 키워드 추출하는 함수 (데이터 나눠서 크롤링하는 함수에 넣어줌) get_keyword(review_list) 이지가 리스트로 받기도 가능하다 했으니
5. 리뷰 내용 요약 함수(매개변수: 분기/반기 별로 나눠진 리뷰 리스트) (크롤링하는 함수에 넣어줌)
6. 이모든걸 실행해주는 주요 함수: review_sum(all_comp_name):


In [8]:
def get_sum_uid(): # 필요할지 안필요할지 미지수 딕셔너리에 다 넣어줄수도 있음 그냥
    pass

In [9]:
def get_year_list(split_data): #분기/반기별 분할된 데이터를 그룹핑할 기준인 연도리스트 얻음
    i = 0
    year_list = [] #년도 개수 체크
    for item in range(len(split_data)):
        year = split_data[item][4][:4]  # YYYY 형식의 연도 추출
        if year not in year_list:
            year_list.append(year) #한 분기/반기에 포함돼있는 연도 리스트 [2023 ,1992...]
    return year_list #이걸로 그루핑 하고 uid 얻을때도 쓰고



In [116]:
sql = ' SELECT comp_uid, review_cont, review_senti_orig, review_rate, review_date ' #senti는 나중에  pred한걸로 수정 요약내용,감성평가,리뷰평점,리뷰날짜
sql += " FROM comp_review "
sql += f" WHERE review_date LIKE '____01' OR review_date LIKE '____02' OR review_date LIKE '____03'; " #1분기의 리뷰 데이터

#1분기 데이터 연도 구분 x
quater_one_data = sc.conn_and_exec(sql)
quater_one_data

((1,
  '온보딩을 기초로 입사부터 체계적으로 회사에 적응할 수 있는 환경을 만들어 줍니다! 다른 회사에서 좋았던 복지가 하나로 뭉친 느낌으로 많은 혜택을 받고 있습니다 :D (그중 유연출퇴근제도 너무 좋아요) 자기계발 할 수 있는 기회가 많아 업무와 연계하여 공부하고 있습니다!',
  'P',
  5,
  '202302'),
 (1,
  '똑똑한 팀원들, 수평적인 업무 분위기, 도움이 필요할 때 손을 내미면 다들 적극적으로 도와주심, 성장의 기회가 많음, 복지가 좋음, 분위기가 밝고 대체로 화기애애함. 의사결정이 투명하게 이루어짐.',
  'P',
  5,
  '202301'),
 (1, '매출 규모가 작음. 다양한 거래처 및 영업 확장이 필요합니다. 그 외 딱히 단점은 없습니다.', 'N', 5, '202302'),
 (1,
  '딱히 떠오르는 것은 없음. 하나 꼽자면 재택근무가 불가능. 그런데 유연근무제나 자리비움이 가능해서 급한 일이 있을 때 스케줄을 조절해서 일할 수 있음.',
  'N',
  5,
  '202301'),
 (1,
  '1인 1법인카드가 일단 최고의 장점 입니다. 구성원의 성장에 대한 지원을 아낌없이 해 줍니다. 피드백 세션을 통해서 부족한게 있는지, 현재 하는 업무에서의 어려운 점을 솔직하게 나눌 수 있는 자리 또한 좋습니다.',
  'P',
  4,
  '202202'),
 (1,
  '젊은 회사라서 좋긴 하지만 시니어가 더 많이 필요하다는 것. 다양한 배경을 가진 사람들이 채용 되었으면 좋겠다. 은근히 장비 지원이 짠다.',
  'N',
  4,
  '202202'),
 (1,
  '1. 편안한 분위기       - 자유로운 연차 사용       - 자유로운 출퇴근 (11-4만 지키면 자유롭게 출퇴근 가능)       - 휴식 쓰고 싶을 때 쓸 수 있음    2. 복지       - 개인 법카를 줌,,, 먹고 싶은 음식 자유롭게 먹을 수 있다는 거(제일 좋음)       - 업무 외 자기계발비 지원해줘서 좋음       - 휴

In [10]:
def data_split_by_quater(): #회사이름과 시간분할 기준 이제 회사별로 이걸 해줘야지
        

        sql = ' SELECT comp_uid, review_cont, review_senti_orig, review_rate, review_date ' #senti는 나중에  pred한걸로 수정 요약내용,감성평가,리뷰평점,리뷰날짜
        sql += " FROM comp_review "
        sql += f" WHERE review_date LIKE '____01' OR review_date LIKE '____02' OR review_date LIKE '____03'; " #1분기의 리뷰 데이터

        #1분기 데이터 연도 구분 x
        quater_one_data = sc.conn_and_exec(sql)
        return quater_one_data


        """sql = ' SELECT review_cont, review_senti_orig, review_rate, review_date ' #senti는 나중에  pred한걸로 수정 요약내용,감성평가,리뷰평점,리뷰날짜
        sql += " FROM comp_review "
        sql += " WHERE review_date LIKE '____04' OR review_date LIKE '____05' OR review_date LIKE '____06'; "

        #2분기 데이터 연도 구분 x
        quater_two_data = sc.conn_and_exec(sql)

        sql = ' SELECT review_cont, review_senti_orig, review_rate, review_date ' #senti는 나중에  pred한걸로 수정 요약내용,감성평가,리뷰평점,리뷰날짜
        sql += " FROM comp_review "
        sql += " WHERE review_date LIKE '____07' OR review_date LIKE '____08' OR review_date LIKE '____09'; "

        #3분기 데이터 연도 구분 x
        quater_three_data = sc.conn_and_exec(sql)

        sql = ' SELECT review_cont, review_senti_orig, review_rate, review_date ' #senti는 나중에  pred한걸로 수정 요약내용,감성평가,리뷰평점,리뷰날짜
        sql += " FROM comp_review "
        sql += " WHERE review_date LIKE '____10' OR review_date LIKE '____11' OR review_date LIKE '____12'; "

        #4분기 데이터 연도 구분 x
        quater_four_data = sc.conn_and_exec(sql)"""

        """ #각 분기 애들 연도별로 묶어준리스트
        try:
            quater_one_data[0][0]

        except:
        #각분기 데이터를 튜플 형태로 묶어줌
        quater_data =(quater_one_data,quater_two_data,quater_three_data,quater_four_data)
        return quater_data"""

    #data[0] 리뷰내용/ data[1] 리뷰감성평가/ data[2] 리뷰평점/ data[3] 리뷰날짜









In [11]:
def get_keyword():
    pass #이지 토크나이저 문제 슬랙에 얘기 나오기 전까진

In [98]:
#1. 연도별로 seperate 2. 연도별 seperate 된 애들중 같은 분기끼리는 전부 concat 요약 모델 for문
def sum_review_dict(split_data,year_list): #분기/반기별로 모아놓은 list quater_data[0]이 1분기 [1] [2] [3] 이 각각 분기 [0][0]은 그 분기정보중 한 row [0][0][0]은cont 하나
    split_data_dict ={}
    new_value_list = []
    #combined_sum_review_list = [] #
    i = 0 #얼마나 왔는지 체크용
    for year in year_list:
        split_data_dict[year] = [] #연도별로

    for year in year_list:
        for item in range(len(split_data)):
            if split_data[item][4][:4] == year:
                #split_data_dict[year].append(split_data[item][0]) #comp_uid를 딕셔너리에 넣어줌
                split_data_dict[year].append((split_data[item][0],split_data[item][1])) #분기/반기 리뷰를 또 연도별로 구분해서 담아줌 uid를 어떻게 구분해서 join하지
                #일단 현재 튜플로 comp_uid와 리뷰내용을 묶어놓은 상태 여기서 어떻게 분리해서 요약따로하지

    #이제 이 내용 담긴 딕셔너리를 key별(연도)로 concat 하면 dict안에 연도별 분기별로 구분된 요약내용이 담긴 딕셔너리 완성
    for key,value in split_data_dict.items(): #딕셔너리 안에 있는 연도별 value들
        #for tp in value: #각 연도별 리스트 안에 있는 튜플들(리뷰와 comp_uid)
        combined_sum_review_list = [] # 매 키가 순회할때마다 리스트 비워줌
        merged_tuples = []
        current_key = None
        current_text = ""

        for key, text in value:
            if key == current_key:
                current_text += " " + text
            else:
                if current_key is not None:
                    merged_tuples.append((current_key, current_text))
                current_key = key
                current_text = text

        # 마지막 데이터 처리
        if current_key is not None:
            merged_tuples.append((current_key, current_text)) # 각 연도별 리뷰같은회사끼리 한꺼번에 묶어준 튜플

        value =merged_tuples #각 연도별 같은 회사 묶어준 튜플

        for combined_review_tp in tqdm(value): #각 tp가 생각해보니까 곧 리뷰 합친거잖아
            #combined_review = ''.join(tp[1]) #연도별 리뷰들을 한꺼번에 묶어줌
            combined_sum_review = data_check.get_summary(combined_review_tp[1],'reviews_long') #연도별 분/반기별 묶인 리뷰 요약
            combined_sum_review_list.append(combined_sum_review)
            if combined_review_tp[0]>10: #테스트용 코드
                break #테스트용으로 회사 10개만 확인
        print(i)
        i+=1

        new_value_temp = [(t[0], combined_sum_review_list[i]) for i, t in enumerate(value) if t[0]<10 ] #회사별 요약해준 값과 uid를 리스트로 묶어 다시 value 리스트에 넣어줌
        new_value_list.append(new_value_temp) #각 연도별 회사들의 리뷰 요약 을 리스트의 인덱스로 가지는 리스트 [[(1,~),(2,~)],[]]
        print(new_value_list)
        #이것도 테스트용이라 value가 t[0]<10 까지만 해놨어 나중에 저 if절 빼면됨
    #for key, value in split_data_dict.items():
    #    split_data_dict[key] = new_value_list.pop(0)# 딕셔너리 키들에 한번에 묶어서 요약해준 문자열을 다시 넣어줌
    for key,value in split_data_dict.items():
        if len(new_value_list) == 0: # 원래는 이코드 넣을 필요없는데 테스트코드라 딕셔너리 year에 포함안된회사(10개라) 있어서 길이가 달라짐 그러므로 테스트용
            break
        split_data_dict[key] = new_value_list.pop(0)# 딕셔너리 키들에 한번에 묶어서 요약해준 문자열을 다시 넣어줌

    return split_data_dict # 성공!!! 연도별 분/반기별 회사별 요약본 만들기 이제는 분/반기 그루핑하면 됨

"""    split_data_dict = {} #분기별 리뷰데이터와 compuid 임시로 저장해줄 딕셔너리
    combined_sum_review_list = [] #f리뷰 요약 내용 담아서 value 넣어줄 리스트
    i = 0

    for year in year_list:
        split_data_dict[year] = [] #key는 year, value는 리스트인 딕셔너리 리스트안에 튜플로 comp_uid와 리뷰내용 담을 예정

    for year in year_list:
        for item in range(len(split_data)): 
            if split_data[item][4][:4] == year: #리뷰데이트가 해당 년도(year)에 포함되면
                split_data_dict[year].append((split_data[item][0], split_data[item][1])) #딕셔너리에 comp_uid와 리뷰내용 넣어줌

    for key, value in split_data_dict.items(): #딕셔너리를 순회하면서 같은 comp_uid끼리 합침
        merged_tuples = []
        current_key = None
        current_text = ""

        for comp_uid, review_text in value:
            if comp_uid == current_key:
                current_text += " " + review_text
            else:
                if current_key is not None:
                    merged_tuples.append((current_key, current_text))
                current_key = comp_uid
                current_text = review_text

        if current_key is not None:
            merged_tuples.append((current_key, current_text)) #같은 uid들의 text는 힙쳐져서 생긴 튜플 모음 

        value[:] = [(t[0], combined_sum_review_list[i]) for i, t in enumerate(merged_tuples)]
        #기존에 value값이던 (uid,reviewcont)에서 같은 comp_uid끼리는 리뷰내용 합쳐서 새로운ㄴ value를 넣어줌 딕서녀러의 value도 같이 바뀜 일단 

        for combined_review_tp in tqdm(value): #comp_uid당 합쳐진 리뷰 큰 1개
            combined_sum_review = data_check.get_summary(combined_review_tp[1], 'reviews_long') #value에서 회사 하나당 리뷰 합친거 각각 요약
            combined_sum_review_list.append(combined_sum_review) #요약 내용을 각각 요약 리스트에 담아줌
            if combined_review_tp[0]>10: #임시 테스트용이라 회사 10개씩만 확인해보기
                break
            print(combined_sum_review)"""


'    split_data_dict = {} #분기별 리뷰데이터와 compuid 임시로 저장해줄 딕셔너리\n    combined_sum_review_list = [] #f리뷰 요약 내용 담아서 value 넣어줄 리스트\n    i = 0\n\n    for year in year_list:\n        split_data_dict[year] = [] #key는 year, value는 리스트인 딕셔너리 리스트안에 튜플로 comp_uid와 리뷰내용 담을 예정\n\n    for year in year_list:\n        for item in range(len(split_data)): \n            if split_data[item][4][:4] == year: #리뷰데이트가 해당 년도(year)에 포함되면\n                split_data_dict[year].append((split_data[item][0], split_data[item][1])) #딕셔너리에 comp_uid와 리뷰내용 넣어줌\n\n    for key, value in split_data_dict.items(): #딕셔너리를 순회하면서 같은 comp_uid끼리 합침\n        merged_tuples = []\n        current_key = None\n        current_text = ""\n\n        for comp_uid, review_text in value:\n            if comp_uid == current_key:\n                current_text += " " + review_text\n            else:\n                if current_key is not None:\n                    merged_tuples.append((current_key, current_text))\n                current_key = comp

In [13]:
def db_save():
    pass#get_all_comp_name

In [96]:
#함수들 다 실행해서 db까지 저장해주는 main function
"""
실행 메커니즘
1.리뷰 긁어올 회사를 받아옴
2.분기별/반기별 데이터를 분할
3.각 분할된 데이터에 있는 연도들을 리스트로 뽑아옴
4.그 연도와 분/반기를 보내 내용 요약 딕셔너리로 만들어줌
5.회사 아이디에 이어붙여
"""

def exec_all():
    #all_comp_uid = get_all_comp_uid() #회사이름 가져옴 리뷰테이블에서 가져오면 되니 굳이?

    quater_one_data = data_split_by_quater() #분기별 데이터 분할 1,2,3,4 #나중에는 그냥 분기 4개 for문에 다넣어버려서 코드 길이 줄이자
    #분기도 그냥 dict key 연도뒤에 붙여버리자
    year_list = get_year_list(quater_one_data)
    quater_one_sum_dict = sum_review_dict(quater_one_data,year_list) #연도별 1분기 dict
    return quater_one_sum_dict
    #quater_two_data = data_split_by_quater(all_comp_name,2) #분기별 데이터 분할 1,2,3,4
    #quater_three_data = data_split_by_quater(all_comp_name,3) #분기별 데이터 분할 1,2,3,4
    #quater_four_data = data_split_by_quater(all_comp_name,4) #분기별 데이터 분할 1,2,3,4



    #sum_review(quater_one_data,quater_two_data,quater_three_data,quater_four_data) #분기,반기 일단 분기만 했는데 내일 요거 토대로 반기도 넣어야됨

    #db_save(sum_review) #각각의 얻은 디비에 넣을 값들 바탕으로 매개변수 받아서 db세이브


In [99]:
year_split_review_dict =exec_all()
year_split_review_dict

  1%|          | 7/856 [00:31<1:03:00,  4.45s/it]


0
[[(1, '온보딩을 기초로 회사에 적응할 수 있는 환경을 만들어 주는 유연출퇴근제도는 업무와 연계하여 공부하고 있다.'), (5, '연차 대비 업계 최고 연봉 수준 높은 복지와 함께 일하는 동료가 좋고 서로 존중하는 문화가 형성되어 있음 글로벌 본사의 우수한 인재들과 일을 하면서 많은 경험을 할 수 있음 복지 워라밸 좋다.'), (6, '금융보안의 전반적인 업무를 담당하고 있어 순환근무를 통해 다양한 직무를 경험해볼 수 있다.'), (7, '기업의 안정성과 고용의 안정성 안정성 자유로운 연차 사용 만족스러운 복지와 연봉은 업무 강도에 비해 많이 받는다고 생각합니다.'), (8, '영업을 배우기에는 괜찮은 곳으로 능력에 따라 인센티브가 높고 영업을 배우기에는 좋은 곳이다.'), (9, '자율성이 부여되는 업무 환경과 사일로 없는 동료들이 존재하고 연월차 무제한이나 좋은 근무환경은 덤 연차 사용 자유로움 채용 절차가 굉장히 오래걸림 업무의 확장성이 굉장히 제한됨 채용 절차가 오래걸림 업무의 확장성이 매우 제한됨 업무의 확장성이 굉장히 제한됨 업무의 확장성이 굉장히 제한됨 업무의 확장성이 굉장히 제한됨 업무의 확장성이 굉장히 제한됨')]]


  1%|          | 9/873 [00:44<1:10:38,  4.91s/it]


1
[[(1, '온보딩을 기초로 회사에 적응할 수 있는 환경을 만들어 주는 유연출퇴근제도는 업무와 연계하여 공부하고 있다.'), (5, '연차 대비 업계 최고 연봉 수준 높은 복지와 함께 일하는 동료가 좋고 서로 존중하는 문화가 형성되어 있음 글로벌 본사의 우수한 인재들과 일을 하면서 많은 경험을 할 수 있음 복지 워라밸 좋다.'), (6, '금융보안의 전반적인 업무를 담당하고 있어 순환근무를 통해 다양한 직무를 경험해볼 수 있다.'), (7, '기업의 안정성과 고용의 안정성 안정성 자유로운 연차 사용 만족스러운 복지와 연봉은 업무 강도에 비해 많이 받는다고 생각합니다.'), (8, '영업을 배우기에는 괜찮은 곳으로 능력에 따라 인센티브가 높고 영업을 배우기에는 좋은 곳이다.'), (9, '자율성이 부여되는 업무 환경과 사일로 없는 동료들이 존재하고 연월차 무제한이나 좋은 근무환경은 덤 연차 사용 자유로움 채용 절차가 굉장히 오래걸림 업무의 확장성이 굉장히 제한됨 채용 절차가 오래걸림 업무의 확장성이 매우 제한됨 업무의 확장성이 굉장히 제한됨 업무의 확장성이 굉장히 제한됨 업무의 확장성이 굉장히 제한됨 업무의 확장성이 굉장히 제한됨')], [(1, '1인 1법인카드는 구성원의 성장에 대한 지원을 해주고 피드백 세션을 통해 부족한 점을 솔직하게 나눌 수 있는 자리도 좋다.'), (2, '기업 문화는 최고의 기업 문화를 갖고 있다.'), (3, 'okr을 통해 성과보상이 높음 복지가 높고 매출에 집중을 많이 하고 있다.'), (5, '처우 및 복지가 좋은 편이고 같이 일하는 다른 국가 직원들이 많아 성장할 수 있는 곳이다.'), (6, '회사의 보수 및 복지 등은 모두 만족하고 있지만 보수와 복지 면에서 뒤쳐지고 있어 보통의 회사가 되었음을 알 수 있다.'), (7, '근무환경과 복지제도 수평적 개방적인 태도와 협조요청에 적극적인 직원들에게 복지 좋다는 회사의 모든 장점은 다 가지고 있다.'), (8, '직원들이 목소리를 내면 들어주려는 노력을 하는 회사 단순 보

  1%|          | 6/752 [00:29<1:01:21,  4.94s/it]


...

  1%|          | 6/892 [00:30<1:15:21,  5.10s/it]


7
[[(1, '온보딩을 기초로 회사에 적응할 수 있는 환경을 만들어 주는 유연출퇴근제도는 업무와 연계하여 공부하고 있다.'), (5, '연차 대비 업계 최고 연봉 수준 높은 복지와 함께 일하는 동료가 좋고 서로 존중하는 문화가 형성되어 있음 글로벌 본사의 우수한 인재들과 일을 하면서 많은 경험을 할 수 있음 복지 워라밸 좋다.'), (6, '금융보안의 전반적인 업무를 담당하고 있어 순환근무를 통해 다양한 직무를 경험해볼 수 있다.'), (7, '기업의 안정성과 고용의 안정성 안정성 자유로운 연차 사용 만족스러운 복지와 연봉은 업무 강도에 비해 많이 받는다고 생각합니다.'), (8, '영업을 배우기에는 괜찮은 곳으로 능력에 따라 인센티브가 높고 영업을 배우기에는 좋은 곳이다.'), (9, '자율성이 부여되는 업무 환경과 사일로 없는 동료들이 존재하고 연월차 무제한이나 좋은 근무환경은 덤 연차 사용 자유로움 채용 절차가 굉장히 오래걸림 업무의 확장성이 굉장히 제한됨 채용 절차가 오래걸림 업무의 확장성이 매우 제한됨 업무의 확장성이 굉장히 제한됨 업무의 확장성이 굉장히 제한됨 업무의 확장성이 굉장히 제한됨 업무의 확장성이 굉장히 제한됨')], [(1, '1인 1법인카드는 구성원의 성장에 대한 지원을 해주고 피드백 세션을 통해 부족한 점을 솔직하게 나눌 수 있는 자리도 좋다.'), (2, '기업 문화는 최고의 기업 문화를 갖고 있다.'), (3, 'okr을 통해 성과보상이 높음 복지가 높고 매출에 집중을 많이 하고 있다.'), (5, '처우 및 복지가 좋은 편이고 같이 일하는 다른 국가 직원들이 많아 성장할 수 있는 곳이다.'), (6, '회사의 보수 및 복지 등은 모두 만족하고 있지만 보수와 복지 면에서 뒤쳐지고 있어 보통의 회사가 되었음을 알 수 있다.'), (7, '근무환경과 복지제도 수평적 개방적인 태도와 협조요청에 적극적인 직원들에게 복지 좋다는 회사의 모든 장점은 다 가지고 있다.'), (8, '직원들이 목소리를 내면 들어주려는 노력을 하는 회사 단순 보

  0%|          | 2/520 [00:11<50:17,  5.83s/it]

8
[[(1, '온보딩을 기초로 회사에 적응할 수 있는 환경을 만들어 주는 유연출퇴근제도는 업무와 연계하여 공부하고 있다.'), (5, '연차 대비 업계 최고 연봉 수준 높은 복지와 함께 일하는 동료가 좋고 서로 존중하는 문화가 형성되어 있음 글로벌 본사의 우수한 인재들과 일을 하면서 많은 경험을 할 수 있음 복지 워라밸 좋다.'), (6, '금융보안의 전반적인 업무를 담당하고 있어 순환근무를 통해 다양한 직무를 경험해볼 수 있다.'), (7, '기업의 안정성과 고용의 안정성 안정성 자유로운 연차 사용 만족스러운 복지와 연봉은 업무 강도에 비해 많이 받는다고 생각합니다.'), (8, '영업을 배우기에는 괜찮은 곳으로 능력에 따라 인센티브가 높고 영업을 배우기에는 좋은 곳이다.'), (9, '자율성이 부여되는 업무 환경과 사일로 없는 동료들이 존재하고 연월차 무제한이나 좋은 근무환경은 덤 연차 사용 자유로움 채용 절차가 굉장히 오래걸림 업무의 확장성이 굉장히 제한됨 채용 절차가 오래걸림 업무의 확장성이 매우 제한됨 업무의 확장성이 굉장히 제한됨 업무의 확장성이 굉장히 제한됨 업무의 확장성이 굉장히 제한됨 업무의 확장성이 굉장히 제한됨')], [(1, '1인 1법인카드는 구성원의 성장에 대한 지원을 해주고 피드백 세션을 통해 부족한 점을 솔직하게 나눌 수 있는 자리도 좋다.'), (2, '기업 문화는 최고의 기업 문화를 갖고 있다.'), (3, 'okr을 통해 성과보상이 높음 복지가 높고 매출에 집중을 많이 하고 있다.'), (5, '처우 및 복지가 좋은 편이고 같이 일하는 다른 국가 직원들이 많아 성장할 수 있는 곳이다.'), (6, '회사의 보수 및 복지 등은 모두 만족하고 있지만 보수와 복지 면에서 뒤쳐지고 있어 보통의 회사가 되었음을 알 수 있다.'), (7, '근무환경과 복지제도 수평적 개방적인 태도와 협조요청에 적극적인 직원들에게 복지 좋다는 회사의 모든 장점은 다 가지고 있다.'), (8, '직원들이 목소리를 내면 들어주려는 노력을 하는 회사 단순 보

{'2023': [(1, '온보딩을 기초로 회사에 적응할 수 있는 환경을 만들어 주는 유연출퇴근제도는 업무와 연계하여 공부하고 있다.'),
  (5,
   '연차 대비 업계 최고 연봉 수준 높은 복지와 함께 일하는 동료가 좋고 서로 존중하는 문화가 형성되어 있음 글로벌 본사의 우수한 인재들과 일을 하면서 많은 경험을 할 수 있음 복지 워라밸 좋다.'),
  (6, '금융보안의 전반적인 업무를 담당하고 있어 순환근무를 통해 다양한 직무를 경험해볼 수 있다.'),
  (7,
   '기업의 안정성과 고용의 안정성 안정성 자유로운 연차 사용 만족스러운 복지와 연봉은 업무 강도에 비해 많이 받는다고 생각합니다.'),
  (8, '영업을 배우기에는 괜찮은 곳으로 능력에 따라 인센티브가 높고 영업을 배우기에는 좋은 곳이다.'),
  (9,
   '자율성이 부여되는 업무 환경과 사일로 없는 동료들이 존재하고 연월차 무제한이나 좋은 근무환경은 덤 연차 사용 자유로움 채용 절차가 굉장히 오래걸림 업무의 확장성이 굉장히 제한됨 채용 절차가 오래걸림 업무의 확장성이 매우 제한됨 업무의 확장성이 굉장히 제한됨 업무의 확장성이 굉장히 제한됨 업무의 확장성이 굉장히 제한됨 업무의 확장성이 굉장히 제한됨')],
 '2022': [(1,
   '1인 1법인카드는 구성원의 성장에 대한 지원을 해주고 피드백 세션을 통해 부족한 점을 솔직하게 나눌 수 있는 자리도 좋다.'),
  (2, '기업 문화는 최고의 기업 문화를 갖고 있다.'),
  (3, 'okr을 통해 성과보상이 높음 복지가 높고 매출에 집중을 많이 하고 있다.'),
  (5, '처우 및 복지가 좋은 편이고 같이 일하는 다른 국가 직원들이 많아 성장할 수 있는 곳이다.'),
  (6, '회사의 보수 및 복지 등은 모두 만족하고 있지만 보수와 복지 면에서 뒤쳐지고 있어 보통의 회사가 되었음을 알 수 있다.'),
  (7, '근무환경과 복지제도 수평적 개방적인 태도와 협조요청에 적극적인 직원들에게 복지 좋다는 회사의 모든 장점은 다 가지

In [89]:
year_split_review_dict['2019']

[(4,
  '3년 정도 꾸준히 배우면 컨설팅이건 현업이건 매우 많은 경력기회를 얻을 수 있음. 시니어 매니지먼트들의 통찰력과 노하우를 배울 수 있음'),
 (4,
  '매 월 인턴 대상 교육이 있으며, &#39;인사&#39;의 아웃라인을 잡거나, 특정 주제의 액션 러닝을 직접 주관하시는 등 교육 의지가 남다르심  경우에 따라 다르나, 실제로 보고서를 직접 작성해 볼 기회를 주시기도 함'),
 (4,
  '컨설팅 회사가 다 그렇듯이 클라이언트에게 보고할 시점이 되면 업무량이 다소 많아지며, 야근이나 주말 근무를 해야 할 때가 있음. 일 잘 하는 사람이면 워라밸에 거의 불만이 없지만 (자기가 잘 조절해서 하면...) 일 못 하면 꽤 고생을 할 수도...'),
 (4,
  '단점일까? 상대적으로 회식이 잦음. 그래도 늦게까지 먹지 않고 일찍 끝나며, 약속 있다고 하면 쿨하게 보내주심. 그런데 매일 쿨하게 가기도 눈치가 보임 ㅎㅎ;'),
 (5, '근무환경이 좋으며 분위기가 굉장히 자유롭고, 내 일에 집중할 수 있어서 좋은 곳입니다.'),
 (5, '분위기좋고 확실한 보상과 수직적이지 않는 분위기가 최고입니다'),
 (5, '복지 복지 음료수 간식거리 공짜 좋은 환경의 오피스 괜찬은 연봉 등등등.....................'),
 (5, '윗사람 눈치 안보고 연차 사용, 점심 저녁 제공, 깔끔한 사무실 분위기 등등'),
 (5, '딱히 꼽을 것이 없지만 굳이 한가지를 꼽자면 컴퓨터를 보고 하는 일이기 때문에 본인의 습관에 따라 눈이 피로할 수도 있습니다.'),
 (5, '단점 그다지 별로 없는거같은데 굳이 생각하면 드가기가 너무 힘드네여'),
 (5, '계약직과 정규직의 차별이 있는 편이다. 대우면에서 나 평소에도................'),
 (5, '낙하산 인사, 인맥... 너무 괴롭다. 일좀 하자... 제발...'),
 (5,
  '발빠른 판단과 협업이 이루어질 수 있는 분위기를 조성합니다. 또한 결과 성취에 너무 많은 비중을 두고 진행되는 프로

In [25]:
combined_data = []
current_key = None
current_text = ""

for review_list in year_split_review_dict.values():
    for review_tuple in review_list:
        key, text = review_tuple
        if key == current_key:
            current_text += " " + text
        else:
            if current_key is not None:
                combined_data.append((current_key, current_text))
            current_key = key
            current_text = text

    # 마지막 데이터 처리
    if current_key is not None:
        combined_data.append((current_key, current_text))

In [31]:
merged_tuples = []
current_key = None
current_text = ""

for key, text in year_split_review_dict['2023']:
    if key == current_key:
        current_text += " " + text
    else:
        if current_key is not None:
            merged_tuples.append((current_key, current_text))
        current_key = key
        current_text = text

# 마지막 데이터 처리
if current_key is not None:
    merged_tuples.append((current_key, current_text))

In [34]:
year_split_review_dict['2023'] =merged_tuples

In [ ]:
for tp in year_split_review_dict['2023']:
    print(tp[0])